In [11]:
dir_webdriver = 'C:\\Neal\\Tufts_University\\capstone\\EDGAR'
dir_html = 'C:\\Neal\\Tufts_University\\capstone\\EDGAR\\2022.html'
import base64
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException

In [20]:



import itertools

driver = webdriver.Firefox(dir_webdriver)

driver.get(dir_html)

found_text = driver.find_elements(By.TAG_NAME, 'span')
# found_annotation = driver.find_elements(By.TAG_NAME, 'ix:nonnumeric')
temp_txt = [i for i in found_text if i.text not in set([' ', ''])]
annotation_dict = dict()
# have_annotation = np.zeros_like(temp_txt, 'bool')
# elements_with_annotation = []
for i in range(len(temp_txt)):
    found_annotation = temp_txt[i].find_elements(By.TAG_NAME, 'ix:nonnumeric')
    found_annotation += temp_txt[i].find_elements(By.TAG_NAME, 'ix:nonfraction')
    # if len(found_annotation) != 0:
        # have_annotation[i] = True
        # elements_with_annotation += [temp_txt[i]]
    annotation_dict[temp_txt[i]] = found_annotation
    





# temp_anno = [i for i in found_annotation if i.text not in set([' ', ''])]

def border_red(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid red; display: inline-block")

def border_blue(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid blue; display: inline-block")
def border_green(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid green; display: inline-block")
def border_yellow(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid yellow; display: inline-block")
def border_pink(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid pink; display: inline-block")


for i in temp_txt:
    border_red(i, driver);
for i in annotation_dict:
    for j in annotation_dict[i]:
        border_blue(j, driver)


# tables = [i.get_attribute('innerHTML') for i in driver.find_elements(By.TAG_NAME, 'table')]
found_table = driver.find_elements(By.TAG_NAME, 'table')


table_is_numeric = np.zeros_like(found_table, 'int')# 0: numerical, 1: non-numerical, 2: unannotated
for i in range(len(found_table)):
    try: 
        found_numeric = found_table[i].find_element(By.TAG_NAME, 'ix:nonfraction')
        table_is_numeric[i] = 0
    except NoSuchElementException:
        try:
            found_numeric = found_table[i].find_element(By.TAG_NAME, 'ix:nonnumeric')
            table_is_numeric[i] = 1
        except NoSuchElementException:
            table_is_numeric[i] = 2

for i in range(len(found_table)):
    if table_is_numeric[i] == 0:
        
        border_green(found_table[i], driver)
    elif table_is_numeric[i] == 1:
        border_yellow(found_table[i], driver)
    else:
        border_pink(found_table[i], driver)

C:\Users\Zhaohui Zhang\AppData\Local\Temp\ipykernel_15288\3509480561.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(dir_webdriver)


In [ ]:
# found_table = driver.find_elements(By.TAG_NAME, 'table')
# found_table[0].get_attribute('innerHTML')

In [5]:
# driver = webdriver.Firefox(dir_webdriver)

# driver.get(dir_html)
# a = driver.find_element(By.TAG_NAME, 'dwfewf')

In [9]:
A = {1:[2,3,4,5], 2:[4,5,6,7]}
for i in A:
    for j in A[i]:
        print(j)

2
3
4
5
4
5
6
7
